In [1]:
# Импорт библиотек

import os # Для взаимодействия с ОС
import shutil # Для управления файлами и папками
import pandas as pd # Для работы с табличными данными
from sklearn.preprocessing import OneHotEncoder # Для преобразования категориальных переменных

## Copying to the Repository

In [2]:
def copying_to_repository():
    # Пути к коренной директории и директории с репозиторием
    source_folder = input('Введите коренную директорию - ')
    git_folder = input('Введите директорию репозитория - ')
    
    # Создание целевой директории
    os.makedirs(git_folder, exist_ok = True)
    
    # Копирование файлов из исходной директории в репозиторий
    for filename in os.listdir(source_folder):
        source_path = os.path.join(source_folder, filename) # Полный путь к файлу в корневой директории
        git_path = os.path.join(git_folder, filename) # Полный путь к директории с репозиторием
    
        if filename != '.ipynb_checkpoints': # Исключение папки при копировании 
            if os.path.isdir(source_path): # Если это директория, то копируем её с содержимым
                shutil.rmtree(git_path, ignore_errors = True) # Удаление старой директории   
                shutil.copytree(source_path, git_path) # Копирование директории
            else: # Иначе копируем файл
                shutil.copy2(source_path, git_path) # Копирование файла
    
    print('Файлы успешно скопированы в репозиторий')

In [37]:
# copying_to_repository()

## Data Preparation

### General Information About the Dataset

In [4]:
# Загрузка датасета

df = pd.read_excel('data/characteristics_and_sales_of_goods.xlsx')
print(f'Размер датасета: {df.shape}')
print('Первые пять записей датасета:')
df.head()

Размер датасета: (20596, 5)
Первые пять записей датасета:


D:\Anaconda\Lib\site-packages\openpyxl\worksheet\_read_only.py:81: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
D:\Anaconda\Lib\site-packages\openpyxl\worksheet\_read_only.py:81: UserWarning: Conditional Formatting extension is not supported and will be removed
  for idx, row in parser.parse():


,product_name,product_category,total_realization,share_realization,accumulation_realization
0,Аленка (ОК) 90г,Шоколад,71201224.45,0.011562,0.011562
1,РОЛЛТОН ВЕРМИШЕЛЬ б/п витамин. куриная на дома...,Продукты быстрого приготовления,64360762.22,0.010451,0.022012
2,"МАКФА МУКА пшеничная в/с, 2кг.Челябинск",Мука,49262315.82,0.007999,0.030012
3,"МАКФА Спираль макар.изд., в/с, 223-3А 400гр.",Макаронные изделия,47573899.91,0.007725,0.037737
4,В шок. Москвичка БК,Карамель,32893871.30,0.005341,0.043078


In [5]:
print('Описательная статистика датасета:')
df.describe()

Описательная статистика датасета:


,total_realization,share_realization,accumulation_realization
count,2.059600e+04,2.059600e+04,20596.000000
mean,2.990104e+05,4.855312e-05,0.942684
std,1.527611e+06,2.480524e-04,0.125434
min,-3.493943e+05,-5.673441e-05,0.011562
25%,8.584725e+02,1.393982e-07,0.954281
50%,1.295191e+04,2.103123e-06,0.996730
75%,1.296487e+05,2.105227e-05,1.000530
max,7.120122e+07,1.156161e-02,1.000771


In [6]:
print('Информация о датасете:')
df.info()

Информация о датасете:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20596 entries, 0 to 20595
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   product_name              20596 non-null  object 
 1   product_category          20596 non-null  object 
 2   total_realization         20596 non-null  float64
 3   share_realization         20596 non-null  float64
 4   accumulation_realization  20596 non-null  float64
dtypes: float64(3), object(2)
memory usage: 804.7+ KB


Пропущенных значений в датасете **нет**

In [7]:
print('Количество уникальных значений в датасете:')
for name, value in df.items():
    print(f'Характеристика: {name}')
    print(f'Количество уникальных значений: {df[name].nunique()}')
    print(f'Список значений:\n{df.groupby(df[name].tolist()).size().sort_values(ascending = False)}')

Количество уникальных значений в датасете:
Характеристика: product_name
Количество уникальных значений: 20451
Список значений:
Казахстанский 100г (БС)                                                                3
Мармелад Ассорти                                                                       3
Салфетки влажные PARLO 120шт универсальные для всей семьи с крышкой                    2
Сыр Шушенский Фигурный коса копченая Семенишна бзмж 45% 100гр                          2
Парижанка французкий шоколадный трюфель для дет.питания                                2
                                                                                      ..
Княжеские SRP-T                                                                        1
Княжеские 200г                                                                         1
Книга путешественника №84                                                              1
Книга путешественника №77                                               

In [8]:
# Привидение текстовых переменных к нижнему регистру

df[['product_name', 'product_category']] = df[['product_name', 'product_category']].apply(lambda x: x.str.lower())
df.head()

,product_name,product_category,total_realization,share_realization,accumulation_realization
0,аленка (ок) 90г,шоколад,71201224.45,0.011562,0.011562
1,роллтон вермишель б/п витамин. куриная на дома...,продукты быстрого приготовления,64360762.22,0.010451,0.022012
2,"макфа мука пшеничная в/с, 2кг.челябинск",мука,49262315.82,0.007999,0.030012
3,"макфа спираль макар.изд., в/с, 223-3а 400гр.",макаронные изделия,47573899.91,0.007725,0.037737
4,в шок. москвичка бк,карамель,32893871.30,0.005341,0.043078


### Cleaning the Dataset

In [9]:
# Вывод товаров с отрицательной суммой реализации

df[df['total_realization'] <= 0]

,product_name,product_category,total_realization,share_realization,accumulation_realization
20138,хилтоп (ж/б) 125г мишка бордовый (копилка),чай,0.00,0.000000,1.000771
20139,хилтоп (ж/б) 125г мишка золотой (копилка),чай,0.00,0.000000,1.000771
20140,упаковка домик для гнома,новогодняя упаковка,0.00,0.000000,1.000771
20141,тушь для ресниц golden rose classics xl volume...,"косметика, гигиена",0.00,0.000000,1.000771
20142,"ультрамарин щука натуральная с д/м, с/кл , 240гр",консервы рыбные и морепродукты,0.00,0.000000,1.000771
...,...,...,...,...,...
20591,"нектар апельсиновый тм мой 0,95л","соки, напитки",-292397.26,-0.000047,1.000211
20592,"нектар вишнёво-яблочный тм мой 0,95л","соки, напитки",-298327.66,-0.000048,1.000163
20593,"нектар яблочно-абрикосовый с мякотью тм мой 0,95л","соки, напитки",-309930.97,-0.000050,1.000113
20594,нива ул/э бк2,конфеты,-344766.28,-0.000056,1.000057


In [10]:
# Удаление товаров с отрицательной суммой реализации

df_clean = df.loc[df['total_realization'] > 0]
print(f'Размер отчищенного датасета - {df_clean.shape}')

Размер отчищенного датасета - (20138, 5)


In [11]:
# Словарь для объединения категорий

category_mapping = {
    'конфеты': ['конфеты', 'конфеты фасованные', 'конфеты в коробках'],
    'шоколад': ['шоколад', 'шоколад весовой', 'шоколодные и ореховые пасты'],
    'печенье': ['печенье', 'печенье фасованное', 'крекер'],
    'вафли': ['вафли', 'вафли фасованные'],
    'карамель': ['карамель', 'карамель фасованная'],
    'чай/кофе': ['чай', 'хорека-чай'],
    'кофе': ['кофе', 'хорека-кофе'],
    'консервы': ['консервы мясные', 'консервы овощные', 'консервы рыбные и морепродукты', 'консервы фруктово-ягодные'],
    'молоко сгущенное': ['молоко сгущенное и концентрированное'],
    'мясные продукты': ['деликатесы мясные', 'мясо', 'мясо охлажденное', 'холодец'],
    'полуфабрикаты': ['полуфабрикаты', 'готовые блюда', 'паназиатская кухня'],
    'хлебобулочные изделия': ['хлеб, хлебобулочные изделия'],
    'к чаю': ['баранки, соломка, сушка, сухари', 'кексы', 'рулеты', 'зефир', 'мармелад', 'пастила', 'восточные сладости', 'козинаки', 'халва'],
    'десерты': ['десерты', 'торты ,бисквиты,пирожные', 'хорека-коржи, торты'],
    'напитки': ['соки, напитки'],
    'крупы и зерновые': ['крупы', 'каши'],
    'растительные масла': ['масла растительные'],
    'соусы и приправы': ['кетчуп и соусы', 'томатная паста'],
    'приправы': ['специи, приправы', 'смеси для гарнира'],
    'продукты для выпечки': ['сода, уксус, дрожжи'],
    'сухие завтраки': ['сухие завтраки', 'хлопья'],
    'замороженные продукты': ['замороженные овощи, ягоды, грибы'],
    'овощи/фрукты': ['овощи,фрукты грибы,ягоды'],
    'детские товары': ['детское питание', 'детские товары'],
    'полезное питание': ['полезное питание', 'полезное питание , продукция без сахара'],
    'снэки': ['снэки', 'жевательная резинка'],
    'новогодние товары': ['новогодние подарки нк', 'новогодние подарки', 'новогодняя упаковка/сайт', 'новогодняя упаковка'],
    'прочее': ['прочие', 'прочие нк(швейный цех)', 'нпу 2024', 'янеиспользуемый', 'остатки сладки', 'элитная продукция', 'подарки'],
    'товары для дома': ['хозяйственные товары', 'бытовая химия'],
    'личная гигиена': ['личная гигиена', 'косметика, гигиена'],
    'табак': ['табак, спички', 'табак собственные магазины'],
    'рыба': ['рыба', 'рыбные пресервы'],
    'яйца': ['яйцо'],
    'орехи/сухофрукты': ['орехи, сухофрукты'],
    'животные': ['товары для животных'],
    'собственные магазины': ['собственные магазины']
}

In [12]:
def replace_categories(df, column_name):
    # Обратный словарь для замены, где ключ - подкатегория, значение - основная категория
    reverse_mapping = {}
    # Заполнение обратного словаря
    # Проход по категориям товаров
    for categories, subcategories in category_mapping.items():
        # Проход по подкатегориям категории товара
        for subcategory in subcategories:
            # Заполнения обратного словаря категорями товаров на основе подкатегории товаров
            reverse_mapping[subcategory] = categories
    # Заполнение стобца датасета категориями из обратного словаря 
    df.loc[:, column_name] = df[column_name].map(reverse_mapping).fillna(df[column_name])
    return df

In [13]:
# Замена категорий товаров

df_clean = replace_categories(df_clean, 'product_category')
df_clean.head()

,product_name,product_category,total_realization,share_realization,accumulation_realization
0,аленка (ок) 90г,шоколад,71201224.45,0.011562,0.011562
1,роллтон вермишель б/п витамин. куриная на дома...,продукты быстрого приготовления,64360762.22,0.010451,0.022012
2,"макфа мука пшеничная в/с, 2кг.челябинск",мука,49262315.82,0.007999,0.030012
3,"макфа спираль макар.изд., в/с, 223-3а 400гр.",макаронные изделия,47573899.91,0.007725,0.037737
4,в шок. москвичка бк,карамель,32893871.30,0.005341,0.043078


In [14]:
# Вывод категорий товаров у которых количество товаров меньше 10

# Группировка категорий товаров и подсчёт количества
category_counts = df_clean.groupby('product_category').size()
# Фильтрация категорий товаров у которых количетсва товаров меньше 10 и сортировка по возрастанию
filtered_category_counts = category_counts[category_counts < 10].sort_values(ascending = False)
# Вывод отфильтрованных категорий товаров
print(f'Категории товаров у которых количество товаров меньше 10:\n{filtered_category_counts}')

Категории товаров у которых количество товаров меньше 10:
product_category
хорека-запчасти                          8
якрупы, мука,сахар                       6
специи/орех                              6
нпу 2023                                 5
яконсервы плодовоовощные                 5
ясоки, газ.вода                          4
мучные изделия                           4
нуга                                     4
хорека-оборудование                      4
мягкая игрушка                           3
нпу 2022                                 3
япродукт быстрого пригот                 2
ямясные консервы                         2
ябакалея                                 2
хорека-минеральная вода                  2
керамика/сайт                            2
новогодние подарки 2022                  2
услуги                                   1
хорека-мороженое                         1
макаронные изделиях (не использовать)    1
рекламные материалы дки                  1
язелень, салаты       

In [15]:
# Удаление категорий товаров у которых количество товаров меньше 10

# Подсчёт количетсва строк для каждой категории товара
category_counts = df_clean['product_category'].value_counts()
# Фильтрация категорий товаров у которых количетсва товаров меньше 10 и определяем их индекс
filtered_category_remove = category_counts[category_counts < 10].index
# Удаление строк с этими категориями
df_clean = df_clean[~df_clean['product_category'].isin(filtered_category_remove)]
print(f'Размер отчищенного датасета - {df_clean.shape}')

Размер отчищенного датасета - (20064, 5)


In [16]:
# Сохранение отчищенного датасета

df_clean.to_excel('data/characteristics_and_sales_of_goods (clean).xlsx')

### Creating a New Feature

In [17]:
# Сортировка по убыванию по сумме реализации

df_new_feature = df_clean.sort_values(by = 'total_realization', ascending = False)
df_new_feature.head()

,product_name,product_category,total_realization,share_realization,accumulation_realization
0,аленка (ок) 90г,шоколад,71201224.45,0.011562,0.011562
1,роллтон вермишель б/п витамин. куриная на дома...,продукты быстрого приготовления,64360762.22,0.010451,0.022012
2,"макфа мука пшеничная в/с, 2кг.челябинск",мука,49262315.82,0.007999,0.030012
3,"макфа спираль макар.изд., в/с, 223-3а 400гр.",макаронные изделия,47573899.91,0.007725,0.037737
4,в шок. москвичка бк,карамель,32893871.30,0.005341,0.043078


In [18]:
# Преобразования накопительной суммы реализации в проценты

df_new_feature['accumulation_realization'] = df_new_feature['accumulation_realization'].apply(lambda x: round((x * 100), 2))
df_new_feature.head()

,product_name,product_category,total_realization,share_realization,accumulation_realization
0,аленка (ок) 90г,шоколад,71201224.45,0.011562,1.16
1,роллтон вермишель б/п витамин. куриная на дома...,продукты быстрого приготовления,64360762.22,0.010451,2.20
2,"макфа мука пшеничная в/с, 2кг.челябинск",мука,49262315.82,0.007999,3.00
3,"макфа спираль макар.изд., в/с, 223-3а 400гр.",макаронные изделия,47573899.91,0.007725,3.77
4,в шок. москвичка бк,карамель,32893871.30,0.005341,4.31


In [19]:
# Функция классификации товаров

def ABC_classification (accumulation_realization):
    if accumulation_realization < 80:
        return 'A'
    elif 80 <= accumulation_realization <= 95:
        return 'B'
    else:
        return 'C'

In [20]:
# Создание нового столбца с ABC классифкацией товаров

df_new_feature['abc_classification'] = df_new_feature['accumulation_realization'].apply(ABC_classification)
df_new_feature.head()

,product_name,product_category,total_realization,share_realization,accumulation_realization,abc_classification
0,аленка (ок) 90г,шоколад,71201224.45,0.011562,1.16,A
1,роллтон вермишель б/п витамин. куриная на дома...,продукты быстрого приготовления,64360762.22,0.010451,2.20,A
2,"макфа мука пшеничная в/с, 2кг.челябинск",мука,49262315.82,0.007999,3.00,A
3,"макфа спираль макар.изд., в/с, 223-3а 400гр.",макаронные изделия,47573899.91,0.007725,3.77,A
4,в шок. москвичка бк,карамель,32893871.30,0.005341,4.31,A


In [21]:
# Вывод количетсва уникальных значений ABC классификации

print(f'Количество уникальных значений: {df_new_feature['abc_classification'].nunique()}')
print(f'Список значений:\n{df_new_feature.groupby(df_new_feature['abc_classification'].tolist()).size().sort_values(ascending = False)}')

Количество уникальных значений: 3
Список значений:
C    15120
B     2987
A     1957
dtype: int64


### Converting Data

In [22]:
# Преобразование 'product_category' из object в category

df_new_feature.product_category = df_new_feature.product_category.astype('category')
print(f'Категории "product_category":')
df_new_feature.product_category.dtype

Категории "product_category":


CategoricalDtype(categories=['алкогольные напитки', 'вафли', 'десерты', 'детские товары',
                  'замороженные продукты', 'к чаю', 'карамель', 'колбасы',
                  'консервы', 'конфеты', 'кофе', 'крупы и зерновые',
                  'личная гигиена', 'майонез', 'макаронные изделия',
                  'масла сливочные', 'молоко сгущенное', 'молочные продукты',
                  'мука', 'мясные продукты', 'мёд', 'напитки',
                  'новогодние товары', 'овощи/фрукты', 'орехи/сухофрукты',
                  'печенье', 'пищевые концентраты', 'полезное питание',
                  'полуфабрикаты', 'приправы',
                  'продукты быстрого приготовления', 'продукты для выпечки',
                  'прочее', 'пряники', 'растворимые напитки',
                  'растительные масла', 'рыба', 'сахар', 'снэки',
                  'собственные магазины', 'соль', 'соусы и приправы',
                  'сухие завтраки', 'сыры', 'табак', 'товары  для животных',
          

In [23]:
# Преобразование 'abc_classification' из object в category

df_new_feature.abc_classification = df_new_feature.abc_classification.astype('category')
print(f'Категории "abc_classification":')
df_new_feature.abc_classification.dtype

Категории "abc_classification":


CategoricalDtype(categories=['A', 'B', 'C'], ordered=False, categories_dtype=object)

In [24]:
print('Информация о датасете:')
df_new_feature.info()

Информация о датасете:
<class 'pandas.core.frame.DataFrame'>
Index: 20064 entries, 0 to 20137
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   product_name              20064 non-null  object  
 1   product_category          20064 non-null  category
 2   total_realization         20064 non-null  float64 
 3   share_realization         20064 non-null  float64 
 4   accumulation_realization  20064 non-null  float64 
 5   abc_classification        20064 non-null  category
dtypes: category(2), float64(3), object(1)
memory usage: 825.5+ KB


In [25]:
# Сохранение датасета с новым параметром

df_new_feature.to_excel('data/characteristics_and_sales_of_goods (new feature).xlsx')

### Transformation of Categorical Variables

In [26]:
# Копирование датасета для дальнейшего использования

df_prepared = df_new_feature.copy()
df_prepared.head()

,product_name,product_category,total_realization,share_realization,accumulation_realization,abc_classification
0,аленка (ок) 90г,шоколад,71201224.45,0.011562,1.16,A
1,роллтон вермишель б/п витамин. куриная на дома...,продукты быстрого приготовления,64360762.22,0.010451,2.20,A
2,"макфа мука пшеничная в/с, 2кг.челябинск",мука,49262315.82,0.007999,3.00,A
3,"макфа спираль макар.изд., в/с, 223-3а 400гр.",макаронные изделия,47573899.91,0.007725,3.77,A
4,в шок. москвичка бк,карамель,32893871.30,0.005341,4.31,A


In [27]:
# Вывод списка значений и их количесвта для категориальных переменных

categories = ['product_category', 'abc_classification']
for categorie in categories:
    unique_count = df_prepared[categorie].nunique()
    print(f'Категория: {categorie}, количество уникальных значений: {unique_count}')

Категория: product_category, количество уникальных значений: 53
Категория: abc_classification, количество уникальных значений: 3


In [28]:
# Создание и вывод датасета с категориальными переменными

categorical_data = df_prepared[categories]
categorical_data.head()

,product_category,abc_classification
0,шоколад,A
1,продукты быстрого приготовления,A
2,мука,A
3,макаронные изделия,A
4,карамель,A


In [29]:
# Создание объекта класса OneHotEncoder, который будет отвечать за преобразование категориальных переменных

ohe = OneHotEncoder(sparse_output = False)
ohe

OneHotEncoder(sparse_output=False)

In [30]:
# Идентифицикация параметров преобразования на датасете с категориальными переменными

ohe.fit(categorical_data[['product_category', 'abc_classification']])

OneHotEncoder(sparse_output=False)

In [31]:
# Примените преобразование для датасета с категориальными переменными и вывод преобразований 
ohe_transform = ohe.fit_transform(categorical_data[['product_category', 'abc_classification']])
ohe_transform

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [32]:
# Вывод наименования признаков 

print(f'Наименование признаков:')
ohe.get_feature_names_out()

Наименование признаков:


array(['product_category_алкогольные напитки', 'product_category_вафли',
       'product_category_десерты', 'product_category_детские товары',
       'product_category_замороженные продукты', 'product_category_к чаю',
       'product_category_карамель', 'product_category_колбасы',
       'product_category_консервы', 'product_category_конфеты',
       'product_category_кофе', 'product_category_крупы и зерновые',
       'product_category_личная гигиена', 'product_category_майонез',
       'product_category_макаронные изделия',
       'product_category_масла сливочные',
       'product_category_молоко сгущенное',
       'product_category_молочные продукты', 'product_category_мука',
       'product_category_мясные продукты', 'product_category_мёд',
       'product_category_напитки', 'product_category_новогодние товары',
       'product_category_овощи/фрукты',
       'product_category_орехи/сухофрукты', 'product_category_печенье',
       'product_category_пищевые концентраты',
       'produ

In [33]:
# Добавление новых признаков в датасет

df_prepared[ohe.get_feature_names_out()] = ohe_transform
df_prepared.head()

,product_name,product_category,total_realization,share_realization,accumulation_realization,abc_classification,product_category_алкогольные напитки,product_category_вафли,product_category_десерты,product_category_детские товары,...,product_category_товары для дома,product_category_хлебобулочные изделия,product_category_хорека-посуда,product_category_хорека-сиропы,product_category_чай/кофе,product_category_шоколад,product_category_яйца,abc_classification_A,abc_classification_B,abc_classification_C
0,аленка (ок) 90г,шоколад,71201224.45,0.011562,1.16,A,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,роллтон вермишель б/п витамин. куриная на дома...,продукты быстрого приготовления,64360762.22,0.010451,2.20,A,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,"макфа мука пшеничная в/с, 2кг.челябинск",мука,49262315.82,0.007999,3.00,A,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,"макфа спираль макар.изд., в/с, 223-3а 400гр.",макаронные изделия,47573899.91,0.007725,3.77,A,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,в шок. москвичка бк,карамель,32893871.30,0.005341,4.31,A,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [34]:
# Удаления колонок датасета 

columns_for_drop = ['product_category', 'total_realization', 'share_realization', 'accumulation_realization', 'abc_classification']
df_prepared = df_prepared.drop(columns = columns_for_drop)
df_prepared.head()

,product_name,product_category_алкогольные напитки,product_category_вафли,product_category_десерты,product_category_детские товары,product_category_замороженные продукты,product_category_к чаю,product_category_карамель,product_category_колбасы,product_category_консервы,...,product_category_товары для дома,product_category_хлебобулочные изделия,product_category_хорека-посуда,product_category_хорека-сиропы,product_category_чай/кофе,product_category_шоколад,product_category_яйца,abc_classification_A,abc_classification_B,abc_classification_C
0,аленка (ок) 90г,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,роллтон вермишель б/п витамин. куриная на дома...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,"макфа мука пшеничная в/с, 2кг.челябинск",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,"макфа спираль макар.изд., в/с, 223-3а 400гр.",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,в шок. москвичка бк,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [35]:
# Сохранение датасета с новым параметром

df_prepared.to_excel('data/characteristics_and_sales_of_goods (prepared).xlsx')